In [ ]:
import numpy as np
import hashlib
import time

from sha3 import sha3_oop, sha3_functional, sha3_numba

def benchmark_and_verify(bit_length, input_bytes, num_iterations):

    # Python SHA3 Implementation (OOP)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        sha3_256 = sha3_oop(bit_length)
        sha3_256.update(input_bytes)
        my_hash = sha3_256.hexdigest()
    my_time = time.perf_counter() - start_time

    # Python SHA3 Implementation (Functional)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        func_hash = sha3_functional(bit_length, input_bytes).hex()
    func_time = time.perf_counter() - start_time

    # Warm up
    sha3_numba(bit_length, input_bytes)
    
    # Python SHA3 Implementation (Functional)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        numba_hash = sha3_numba(bit_length, input_bytes).tobytes().hex()
    numba_time = time.perf_counter() - start_time

    # HashLib SHA3 Implementation
    hashlib_func = hashlib.sha3_256 if bit_length == 256 else hashlib.sha3_512
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        lib_hash = hashlib_func(input_bytes).hexdigest()
    lib_time = time.perf_counter() - start_time
    
    print(f"Input Length: {len(input_bytes)} bytes, Iterations: {num_iterations}")
    print(f"Python  SHA3-{bit_length} Time: {my_time:.4f} s, Hashes/s: {num_iterations/my_time:.1f}, Length: {len(my_hash)}, Hash: {my_hash}")
    print(f"Func.   SHA3-{bit_length} Time: {func_time:.4f} s, Hashes/s: {num_iterations/func_time:.1f}, Length: {len(func_hash)}, Hash: {func_hash}")
    print(f"Numba   SHA3-{bit_length} Time: {numba_time:.4f} s, Hashes/s: {num_iterations/numba_time:.1f}, Length: {len(numba_hash)}, Hash: {numba_hash}")
    print(f"HashLib SHA3-{bit_length} Time: {lib_time:.4f} s, Hashes/s: {num_iterations/lib_time:.1f}, Length: {len(lib_hash)}, Hash: {lib_hash}")

    # Verify Outputs
    assert my_hash == lib_hash, f"Mismatch: {my_hash} != {lib_hash}"
    assert func_hash == lib_hash, f"Mismatch: {func_hash} != {lib_hash}"
    assert numba_hash == lib_hash, f"Mismatch: {numba_hash} != {lib_hash}"
    print("Verification: PASSED\n")


In [ ]:
BIT_LENGTHS = [256, 512] # SHA3-256, SHA3-512
NUM_ITERATIONS = 1000 # Number of iterations to run

for bit_length in BIT_LENGTHS:

    # First, test empty input
    benchmark_and_verify(bit_length, bytes(), NUM_ITERATIONS)

    # Test 'Hello, World!' input
    benchmark_and_verify(bit_length, bytes('Hello, World!', 'utf-8'), NUM_ITERATIONS)

    # Test random 1000 bytes
    benchmark_and_verify(bit_length, np.random.bytes(1000), NUM_ITERATIONS)